In [5]:
import tensorflow as tf

from network import Network

class ResAttentionNet(Network):
    """
    Implementation of Residual Attention Network model
    """
    def setup(self):
        # Pre_conv
        self.pre_conv = self.conv(self.inputs, 3, 3, 16, 1, 1, "pre_conv", relu=False)
        # Attention_1
        self.attention_1 = self.attention_module(self.pre_conv, 16, "attention_1")
        # Trans_1
        self.trans_1 = self.residual_unit(self.attention_1, 16, 16, "trans_2", stride=1)
        # Attention_2
        self.attention_2 = self.attention_module(self.trans_1, 16, "attention_2")
        # Trans_2
        self.trans_2 = self.residual_unit(self.attention_2, 16, 32, "trans_3", stride=2)
        # Attention_3
        self.attention_3 = self.attention_module(self.trans_2, 32, "attention_3")
        # Post_res_1
        self.post_res_1 = self.residual_unit(self.attention_3, 32, 64, "post_res_1", stride=2)
        # Post_res_2
        self.post_res_2 = self.residual_unit(self.post_res_1, 64, 64, "post_res_2")
        # Post_res_3
        self.post_res_3 = self.residual_unit(self.post_res_2, 64, 64, "post_res_3")
        # Post_res_4
        self.post_res_4 = self.residual_unit(self.post_res_3, 64, 64, "post_res_4")
        # Post_bn
        self.post_bn = self.batch_normal(self.post_res_4, self.is_train, "post_bn", tf.nn.relu) 
        # Ave
        self.post_pool = self.avg_pool(self.post_bn, 8, 8, 1, 1, "post_pool")
        # raw_score
        self.raw_score = self.fc(self.post_pool, 2, "raw_score", relu=False)
        # score
        self.score = self.softmax(self.raw_score, "score")

    def attention_module(self, x, ci, name, p=1, t=2, r=1):
        """
        Implementation of Attention Module (2 pool in soft mask branch)
        Input:
        --- x: Module input, 4-D Tensor, with shape [bsize, height, width, channel]
        --- ci: Input channels
        --- name: Module name
        --- p: The number of pre-processing Residual Units
        --- t: The number of Residual Units in trunk branch
        --- r: The number of Residual Units between adjacent pooling layer in the mask branch
        Output:
        --- outputs: Module output
        """
        with tf.name_scope(name), tf.variable_scope(name):
            # Pre-processing Residual Units
            with tf.name_scope("pre_processing"), tf.variable_scope("pre_processing"):
                pre_pros = x
                for idx in range(p):
                    unit_name = "pre_res_{}".format(idx + 1)
                    pre_pros = self.residual_unit(pre_pros, ci, ci, unit_name)
            # Trunk branch
            with tf.name_scope("trunk_branch"), tf.variable_scope("trunk_branch"):
                trunks = pre_pros
                for idx in range(t):
                    unit_name = "trunk_res_{}".format(idx + 1)
                    trunks = self.residual_unit(trunks, ci, ci, unit_name)
            # Mask branch
            with tf.name_scope("mask_branch"), tf.variable_scope("mask_branch"):
                size_1 = pre_pros.get_shape().as_list()[1:3]
                # Max pooling
                masks_1 = self.max_pool(pre_pros, 3, 3, 2, 2, "pool_1")
                for idx in range(r):
                    unit_name = "mask_res1_{}".format(idx + 1)
                    masks_1 = self.residual_unit(masks_1, ci, ci, unit_name)
                size_2 = masks_1.get_shape().as_list()[1:3]
                # Max pooling
                masks_2 = self.max_pool(masks_1, 3, 3, 2, 2, "pool_2")
                for idx in range(2 * r):
                    unit_name = "mask_res2_{}".format(idx + 1)
                    masks_2 = self.residual_unit(masks_2, ci, ci, unit_name)
                # Interpolation
                masks_3 = self.upsample(masks_2, "inter_1", size_2)
                # Skip connection
                skip = self.residual_unit(masks_1, ci, ci, "skip")
                masks_3 = tf.add(masks_3, skip, name="fuse_add")
                for idx in range(r):
                    unit_name = "mask_res3_{}".format(idx + 1)
                    masks_3 = self.residual_unit(masks_3, ci, ci, unit_name)
                # Interpolation
                masks_4 = self.upsample(masks_3, "inter_2", size_1)
                # Batch Normalization
                masks_4 = self.batch_normal(masks_4, self.is_train, "mask_bn1", tf.nn.relu)
                # 1x1 Convolution
                masks_4 = self.conv(masks_4, 1, 1, ci, 1, 1, "mask_conv1", relu=False)
                # Batch Normalization
                masks_4 = self.batch_normal(masks_4, self.is_train, "mask_bn2", tf.nn.relu)
                # 1x1 Convolution
                masks_4 = self.conv(masks_4, 1, 1, ci, 1, 1, "mask_conv2", relu=False)
                # Sigmoid
                masks_4 = tf.nn.sigmoid(masks_4, "mask_sigmoid")
            # Fusing
            with tf.name_scope("fusing"), tf.variable_scope("fusing"):
                outputs = tf.multiply(trunks, masks_4, name="fuse_mul")
                outputs = tf.add(trunks, outputs, name="fuse_add")
            # Post-processing Residual Units
            with tf.name_scope("post_processing"), tf.variable_scope("post_processing"):
                for idx in range(p):
                    unit_name = "post_res_{}".format(idx + 1)
                    outputs = self.residual_unit(outputs, ci, ci, unit_name)
        return outputs

    def residual_unit(self, x, ci, co, name, stride=1):
        """
        Implementation of Residual Unit
        Input:
        --- x: Unit input, 4-D Tensor, with shape [bsize, height, width, channel]
        --- ci: Input channels
        --- co: Output channels
        --- name: Unit name
        --- stride: Convolution stride
        Output:
        --- outputs: Unit output
        """
        with tf.name_scope(name), tf.variable_scope(name):
            # Batch Normalization
            bn_1 = self.batch_normal(x, self.is_train, "bn_1", tf.nn.relu)
            # 1x1 Convolution
            conv_1 = self.conv(bn_1, 1, 1, co/4, 1, 1, "conv_1", relu=False)
            # Batch Normalization
            bn_2 = self.batch_normal(conv_1, self.is_train, "bn_2", tf.nn.relu)
            # 3x3 Convolution
            conv_2 = self.conv(bn_2, 3, 3, co/4, stride, stride, "conv_2", relu=False)
            # Batch Normalization
            bn_3 = self.batch_normal(conv_2, self.is_train, "bn_3", tf.nn.relu)
            # 1x1 Convolution
            conv_3 = self.conv(bn_3, 1, 1, co, 1, 1, "conv_3", relu=False)
            # Skip connection
            if co != ci or stride > 1:
                skip = self.conv(bn_1, 1, 1, co, stride, stride, "conv_skip", relu=False)
            else:
                skip = x
            outputs = tf.add(conv_3, skip, name="fuse")
            return outputs


In [6]:
def losses(logits, labels, name):
    with tf.variable_scope('loss') as scope:
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits \
            (logits=logits, labels=labels, name='xentropy_per_example')
        loss = tf.reduce_mean(cross_entropy, name='loss')
        if name=='train':
            tf.summary.scalar(scope.name + '/train_loss', loss)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_loss', loss)
    return loss
 
def trainning(loss, learning_rate):
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step= global_step)
    return train_op
 
def evaluation(logits, labels, name):
    with tf.variable_scope('accuracy') as scope:
        correct = tf.nn.in_top_k(logits, labels, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_accuracy', accuracy)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_accuracy', accuracy)
    return accuracy

def recall_precision(logits, labels, name):
    logits = tf.cast(logits, tf.int64)
    labels = tf.cast(labels, tf.int64)
    predict = tf.arg_max(logits,1)
    with tf.variable_scope('recall_precision') as scope:
        TP = tf.count_nonzero(predict * labels)
        TN = tf.count_nonzero((predict - 1) * (labels - 1))
        FN = tf.count_nonzero(predict * (labels - 1))
        FP = tf.count_nonzero((predict - 1) * labels)
        precision = tf.divide(TP, TP + FP)
        recall = tf.divide(TP, TP + FN)
        precision = tf.cast(precision, dtype=tf.float64)
        recall = tf.cast(recall, dtype=tf.float64)
        #f1 = 2 * precision * recall / (precision + recall)
        #f1 = tf.cast(f1, dtype=tf.float32)
        if name=='train':
            tf.summary.scalar(scope.name + '/train_precision', precision)
            tf.summary.scalar(scope.name + '/train_recall', recall)
        if name=='valid':
            tf.summary.scalar(scope.name + '/valid_precision', precision)
            tf.summary.scalar(scope.name + '/valid_recall', recall)
    return precision, recall

In [7]:
import os
 
def get_files(path_pos,path_neg,label_pos,label_neg):
    TC = []
    label_TC = []
    nonTC = []
    label_nonTC = []
    # data loader
    file_dir_TC=path_pos
    file_dir_nonTC=path_neg
    TC_list = os.listdir(file_dir_TC)
    nonTC_list = os.listdir(file_dir_nonTC)
    for file in TC_list[:len(nonTC_list)]:
        name = file.split('_')
        if name[0] == label_pos:
            TC.append(file_dir_TC + file)
            label_TC.append(1)
    for file in nonTC_list:
        name = file.split('_')
        if name[0] == label_neg:
            nonTC.append(file_dir_nonTC + file)
            label_nonTC.append(0)
    print("There are %d TC\nThere are %d nonTC" % (len(TC), len(nonTC)))
 
    # shuffle
    image_list = np.hstack((TC, nonTC))
    label_list = np.hstack((label_TC, label_nonTC))
    temp = np.array([image_list, label_list])
    temp = temp.transpose()    
    np.random.shuffle(temp)
 
    image_list = list(temp[:, 0])
    label_list = list(temp[:, 1])
    label_list = [int(i) for i in label_list]
 
    return image_list, label_list
 
# img_list,label_list = get_files(file_dir)
 
# batch
def get_batch(image, label, image_W, image_H, batch_size, capacity):   
    image = tf.cast(image, tf.string)
    label = tf.cast(label, tf.int32)
 
    # queue
    input_queue = tf.train.slice_input_producer([image, label])
 
    image_contents = tf.read_file(input_queue[0])
    label = input_queue[1]
    image = tf.image.decode_jpeg(image_contents, channels=1)
 
    # resize
    image = tf.image.resize_images(image, [image_H, image_W], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = tf.cast(image, tf.float32)
    # image = tf.image.per_image_standardization(image)  
    image_batch, label_batch = tf.train.batch([image, label],
                                              batch_size=batch_size,
                                              num_threads=64,  
                                              capacity=capacity)
  
    return image_batch, label_batch

In [ ]:
import os
import numpy as np
import tensorflow as tf
tf.reset_default_graph()
N_CLASSES = 2 
IMG_W = 64  # resize
IMG_H = 64
BATCH_SIZE = 64
CAPACITY = 2000
MAX_STEP = 20000000 
learning_rate = 0.0001 
 
train_dir = './train_attention_resnet_git/'
logs_train_dir = './train_attention_resnet_git/'

file_dir_TC='/home/ubuntu/data/TC/'
file_dir_nonTC='/home/ubuntu/data/nonTC/'
file_dir_valTC='/home/ubuntu/data/valTC_cp/'
file_dir_valnonTC='/home/ubuntu/data/valnonTC/'

train, train_label = get_files(file_dir_TC,file_dir_nonTC,'TC','nonTC')
valid, valid_label = get_files(file_dir_valTC,file_dir_valnonTC,'valTC','valnonTC')
train_batch_op,train_label_batch_op=get_batch(train,
                                train_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)

valid_batch_op,valid_label_batch_op=get_batch(valid,
                                valid_label,
                                IMG_W,
                                IMG_H,
                                BATCH_SIZE,
                                CAPACITY)
x = tf.placeholder(tf.float32, [BATCH_SIZE,IMG_W,IMG_H,1])
y = tf.placeholder(tf.float32, [BATCH_SIZE])
y = tf.cast(y,tf.int64)

train_net = ResAttentionNet(x)
train_logits = train_net.score
train_loss = losses(train_logits, y, 'train')
train_op = trainning(train_loss, learning_rate)
train_acc = evaluation(train_logits, y, 'train')
train_recall, train_precision = recall_precision(train_logits, y, 'train')

valid_loss = losses(train_logits, y, 'valid')
valid_acc = evaluation(train_logits, y, 'valid')
valid_recall, valid_precision = recall_precision(train_logits, y, 'valid')

summary_op = tf.summary.merge_all() 
 
sess = tf.Session()

train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)

saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
 

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
 
try:
    print('----------training start---------')
    train_result=[]
    valid_result=[]
    for step in np.arange(MAX_STEP):
        train_batch,train_label_batch = sess.run([train_batch_op,train_label_batch_op])
        if coord.should_stop():
                break
       
        _, tra_loss, tra_acc, tra_recall, tra_precision= sess.run([train_op, train_loss, train_acc, train_recall, train_precision],feed_dict={x:train_batch,y:train_label_batch})
        summary_str = sess.run(summary_op,feed_dict={x:train_batch,y:train_label_batch})
        train_writer.add_summary(summary_str, step)
        train_result.append([step, tra_loss, tra_acc*100.0, tra_recall*100.0, tra_precision*100.0])
        if step % 50 == 0:
            print('Step %d, train loss = %.2f, train accuracy = %.2f%%, train recall = %.2f%%, train precision = %.2f%%' %(step, tra_loss, tra_acc*100.0, tra_recall*100.0, tra_precision*100.0))
            
            summary = tf.Summary()
            summary.value.add(tag='tra_loss', simple_value=tra_loss)
            summary.value.add(tag='tra_acc', simple_value=tra_acc)
            summary.value.add(tag='tra_recall', simple_value=tra_recall)
            summary.value.add(tag='tra_precision', simple_value=tra_precision)
            train_writer.add_summary(summary, step)
        if step % 500 == 0:
            ckpt=tf.train.get_checkpoint_state('./train_attention_resnet_git/')
#             print(ckpt)
            if ckpt and ckpt.all_model_checkpoint_paths:
                valid_batch,valid_label_batch = sess.run([valid_batch_op,valid_label_batch_op])
                saver.restore(sess,ckpt.model_checkpoint_path)
                val_loss, val_acc, val_recall, val_precision = sess.run([valid_loss, valid_acc, valid_recall, valid_precision], feed_dict={x:valid_batch,y:valid_label_batch})
                valid_result.append([step, val_loss, val_acc*100.0, val_recall*100.0, val_precision*100.0])
                print('*********************')
                print('Step %d, valid loss = %.2f, valid accuracy = %.2f%%, valid recall = %.2f%%, valid precision = %.2f%%' %(step, val_loss, val_acc*100.0, val_recall*100.0, val_precision*100.0))
                print('*********************')
                summary = tf.Summary()
                summary.value.add(tag='val_loss', simple_value=val_loss)
                summary.value.add(tag='val_acc', simple_value=val_acc)
                summary.value.add(tag='val_recall', simple_value=val_recall)
                summary.value.add(tag='val_precision', simple_value=val_precision)
                train_writer.add_summary(summary, step)
#                 summary_str = sess.run(summary_op,feed_dict={x:valid_batch,y:valid_label_batch})
#                 train_writer.add_summary(summary_str, step)
            
      
        if step % 500 == 0 or (step + 1) == MAX_STEP:
            checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step=step)

except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')

finally:
    coord.request_stop()

There are 1737956 TC
There are 1737956 nonTC
There are 43175 TC
There are 434488 nonTC
Tensor("Placeholder:0", shape=(64, 64, 64, 1), dtype=float32)
[64, 64, 64, 1]
Tensor("attention_1/attention_1/pre_processing/pre_processing/pre_res_1/pre_res_1/bn_1/bn_1/BatchNorm/Relu:0", shape=(64, 64, 64, 16), dtype=float32)
[64, 64, 64, 16]
Tensor("attention_1/attention_1/pre_processing/pre_processing/pre_res_1/pre_res_1/bn_2/bn_2/BatchNorm/Relu:0", shape=(64, 64, 64, 4), dtype=float32)
[64, 64, 64, 4]
Tensor("attention_1/attention_1/pre_processing/pre_processing/pre_res_1/pre_res_1/bn_3/bn_3/BatchNorm/Relu:0", shape=(64, 64, 64, 4), dtype=float32)
[64, 64, 64, 4]
Tensor("attention_1/attention_1/trunk_branch/trunk_branch/trunk_res_1/trunk_res_1/bn_1/bn_1/BatchNorm/Relu:0", shape=(64, 64, 64, 16), dtype=float32)
[64, 64, 64, 16]
Tensor("attention_1/attention_1/trunk_branch/trunk_branch/trunk_res_1/trunk_res_1/bn_2/bn_2/BatchNorm/Relu:0", shape=(64, 64, 64, 4), dtype=float32)
[64, 64, 64, 4]
Tensor

Tensor("attention_2/attention_2/mask_branch/mask_branch/mask_res3_1/mask_res3_1/bn_1/bn_1/BatchNorm/Relu:0", shape=(64, 31, 31, 16), dtype=float32)
[64, 31, 31, 16]
Tensor("attention_2/attention_2/mask_branch/mask_branch/mask_res3_1/mask_res3_1/bn_2/bn_2/BatchNorm/Relu:0", shape=(64, 31, 31, 4), dtype=float32)
[64, 31, 31, 4]
Tensor("attention_2/attention_2/mask_branch/mask_branch/mask_res3_1/mask_res3_1/bn_3/bn_3/BatchNorm/Relu:0", shape=(64, 31, 31, 4), dtype=float32)
[64, 31, 31, 4]
Tensor("attention_2/attention_2/mask_branch/mask_branch/mask_bn1/mask_bn1/BatchNorm/Relu:0", shape=(64, 64, 64, 16), dtype=float32)
[64, 64, 64, 16]
Tensor("attention_2/attention_2/mask_branch/mask_branch/mask_bn2/mask_bn2/BatchNorm/Relu:0", shape=(64, 64, 64, 16), dtype=float32)
[64, 64, 64, 16]
Tensor("attention_2/attention_2/post_processing/post_processing/post_res_1/post_res_1/bn_1/bn_1/BatchNorm/Relu:0", shape=(64, 64, 64, 16), dtype=float32)
[64, 64, 64, 16]
Tensor("attention_2/attention_2/post_pro

Step 150, train loss = 0.53, train accuracy = 76.56%, train recall = 3.12%, train precision = 100.00%
Step 200, train loss = 0.51, train accuracy = 79.69%, train recall = 0.00%, train precision = nan%
Step 250, train loss = 0.52, train accuracy = 76.56%, train recall = 3.45%, train precision = 100.00%
Step 300, train loss = 0.51, train accuracy = 81.25%, train recall = 0.00%, train precision = nan%
Step 350, train loss = 0.53, train accuracy = 82.81%, train recall = 0.00%, train precision = nan%
Step 400, train loss = 0.49, train accuracy = 82.81%, train recall = 0.00%, train precision = nan%
Step 450, train loss = 0.51, train accuracy = 81.25%, train recall = 0.00%, train precision = nan%
Step 500, train loss = 0.42, train accuracy = 89.06%, train recall = 0.00%, train precision = nan%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-0
*********************
Step 500, valid loss = 0.64, valid accuracy = 89.06%, valid recall = 0.00%, valid precision = na

*********************
Step 3500, valid loss = 0.64, valid accuracy = 90.62%, valid recall = 0.00%, valid precision = nan%
*********************
Step 3550, train loss = 0.56, train accuracy = 73.44%, train recall = 0.00%, train precision = nan%
Step 3600, train loss = 0.52, train accuracy = 79.69%, train recall = 3.12%, train precision = 100.00%
Step 3650, train loss = 0.47, train accuracy = 84.38%, train recall = 0.00%, train precision = nan%
Step 3700, train loss = 0.52, train accuracy = 75.00%, train recall = 0.00%, train precision = nan%
Step 3750, train loss = 0.47, train accuracy = 82.81%, train recall = 0.00%, train precision = nan%
Step 3800, train loss = 0.56, train accuracy = 75.00%, train recall = 0.00%, train precision = nan%
Step 3850, train loss = 0.49, train accuracy = 81.25%, train recall = 0.00%, train precision = nan%
Step 3900, train loss = 0.46, train accuracy = 84.38%, train recall = 0.00%, train precision = nan%
Step 3950, train loss = 0.46, train accuracy = 85.94%

Step 6900, train loss = 0.54, train accuracy = 71.88%, train recall = 0.00%, train precision = nan%
Step 6950, train loss = 0.53, train accuracy = 76.56%, train recall = 0.00%, train precision = nan%
Step 7000, train loss = 0.48, train accuracy = 81.25%, train recall = 0.00%, train precision = nan%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-6500
*********************
Step 7000, valid loss = 0.64, valid accuracy = 90.62%, valid recall = 0.00%, valid precision = nan%
*********************
Step 7050, train loss = 0.59, train accuracy = 71.88%, train recall = 0.00%, train precision = nan%
Step 7100, train loss = 0.54, train accuracy = 75.00%, train recall = 2.86%, train precision = 100.00%
Step 7150, train loss = 0.52, train accuracy = 81.25%, train recall = 0.00%, train precision = nan%
Step 7200, train loss = 0.53, train accuracy = 78.12%, train recall = 2.86%, train precision = 100.00%
Step 7250, train loss = 0.56, train accuracy = 71.88%, train re

Step 10200, train loss = 0.63, train accuracy = 67.19%, train recall = 0.00%, train precision = nan%
Step 10250, train loss = 0.57, train accuracy = 73.44%, train recall = 0.00%, train precision = nan%
Step 10300, train loss = 0.56, train accuracy = 75.00%, train recall = 0.00%, train precision = nan%
Step 10350, train loss = 0.54, train accuracy = 73.44%, train recall = 0.00%, train precision = nan%
Step 10400, train loss = 0.51, train accuracy = 81.25%, train recall = 0.00%, train precision = nan%
Step 10450, train loss = 0.55, train accuracy = 73.44%, train recall = 0.00%, train precision = nan%
Step 10500, train loss = 0.46, train accuracy = 85.94%, train recall = 0.00%, train precision = nan%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-10000
*********************
Step 10500, valid loss = 0.63, valid accuracy = 93.75%, valid recall = 0.00%, valid precision = nan%
*********************
Step 10550, train loss = 0.56, train accuracy = 76.56%, trai

*********************
Step 13500, valid loss = 0.64, valid accuracy = 85.94%, valid recall = 0.00%, valid precision = nan%
*********************
Step 13550, train loss = 0.52, train accuracy = 81.25%, train recall = 0.00%, train precision = nan%
Step 13600, train loss = 0.55, train accuracy = 71.88%, train recall = 0.00%, train precision = nan%
Step 13650, train loss = 0.51, train accuracy = 79.69%, train recall = 0.00%, train precision = nan%
Step 13700, train loss = 0.46, train accuracy = 87.50%, train recall = 0.00%, train precision = nan%
Step 13750, train loss = 0.52, train accuracy = 75.00%, train recall = 0.00%, train precision = nan%
Step 13800, train loss = 0.58, train accuracy = 73.44%, train recall = 0.00%, train precision = nan%
Step 13850, train loss = 0.55, train accuracy = 73.44%, train recall = 3.33%, train precision = 100.00%
Step 13900, train loss = 0.54, train accuracy = 75.00%, train recall = 0.00%, train precision = nan%
Step 13950, train loss = 0.57, train accurac

Step 16850, train loss = 0.52, train accuracy = 79.69%, train recall = 0.00%, train precision = nan%
Step 16900, train loss = 0.43, train accuracy = 89.06%, train recall = 0.00%, train precision = nan%
Step 16950, train loss = 0.47, train accuracy = 84.38%, train recall = 0.00%, train precision = nan%
Step 17000, train loss = 0.40, train accuracy = 93.75%, train recall = 0.00%, train precision = nan%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-16500
*********************
Step 17000, valid loss = 0.63, valid accuracy = 95.31%, valid recall = 0.00%, valid precision = nan%
*********************
Step 17050, train loss = 0.54, train accuracy = 79.69%, train recall = 0.00%, train precision = nan%
Step 17100, train loss = 0.48, train accuracy = 81.25%, train recall = 0.00%, train precision = nan%
Step 17150, train loss = 0.59, train accuracy = 70.31%, train recall = 0.00%, train precision = nan%
Step 17200, train loss = 0.57, train accuracy = 71.88%, trai

Step 20100, train loss = 0.50, train accuracy = 82.81%, train recall = 0.00%, train precision = nan%
Step 20150, train loss = 0.51, train accuracy = 79.69%, train recall = 0.00%, train precision = nan%
Step 20200, train loss = 0.55, train accuracy = 76.56%, train recall = 3.70%, train precision = 100.00%
Step 20250, train loss = 0.51, train accuracy = 78.12%, train recall = 0.00%, train precision = nan%
Step 20300, train loss = 0.53, train accuracy = 78.12%, train recall = 2.94%, train precision = 100.00%
Step 20350, train loss = 0.55, train accuracy = 76.56%, train recall = 0.00%, train precision = nan%
Step 20400, train loss = 0.53, train accuracy = 76.56%, train recall = 0.00%, train precision = nan%
Step 20450, train loss = 0.55, train accuracy = 70.31%, train recall = 0.00%, train precision = nan%
Step 20500, train loss = 0.46, train accuracy = 84.38%, train recall = 0.00%, train precision = nan%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-200

Step 23500, train loss = 0.54, train accuracy = 73.44%, train recall = 0.00%, train precision = nan%
INFO:tensorflow:Restoring parameters from ./train_attention_resnet_git/model.ckpt-23000
*********************
Step 23500, valid loss = 0.63, valid accuracy = 92.19%, valid recall = 0.00%, valid precision = nan%
*********************
Step 23550, train loss = 0.54, train accuracy = 79.69%, train recall = 0.00%, train precision = nan%
Step 23600, train loss = 0.49, train accuracy = 82.81%, train recall = 0.00%, train precision = nan%
Step 23650, train loss = 0.53, train accuracy = 78.12%, train recall = 0.00%, train precision = nan%
Step 23700, train loss = 0.52, train accuracy = 75.00%, train recall = 0.00%, train precision = nan%
Step 23750, train loss = 0.49, train accuracy = 84.38%, train recall = 0.00%, train precision = nan%
Step 23800, train loss = 0.57, train accuracy = 70.31%, train recall = 0.00%, train precision = nan%
Step 23850, train loss = 0.43, train accuracy = 89.06%, trai